<H1>Coursera Capstone Project - The Battle of Neighborhoods</H1>


<H2>1. Introduction</H2>
Final project of IBM's Applied Data Science Coursera on the Coursera platform. During the last module, we analyzed data from New York City and Toronto, and used the Foursquare API to get information on recommended places in different areas of both cities.
This project is centered in Madrid, Spain. The objective of the project is to analyze the different neighborhoods of the city and find out which is the best neighborhood to open a specific type of restaurant. Madrid is divided into 21 districts and 131 neighborhoods. To do this, we will take advantage of some open data provided by the Madrid City Council, as well as data from the FourSquare API. The next section will present the different data that will be used throughout the project.

Before we start discussing the data, let's import the packages we need

In [1]:
import requests

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from geopy.geocoders import Nominatim
import folium

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.cluster import KMeans

<H2>2. The data</H2>
The municipality of Madrid has made available to the public more than 400 datasets about different aspects of the city. All data can be found in this website (in Spanish). Within this project, we will use the following datasets from the website:

-Neighborhoods, with area and perimeter (CSV, 10 Kbytes) - Full relation of the 131 neighborhoods in Madrid
Districts, with area and number of neighborhoods (CSV, 1 Kbyte) - Name, area and number of neighborhoods of the 21 districts in Madrid.
-Catalog of venues and their activities (December 2020) (CSV, 60491 Kbytes) - Comprehensive catalog of all venues in Madrid, along with their status (e.g. whether it is open of closed, etc.). We will also get information about the venues in each neighborhood
-Catalog of venues and their activities (December 2019) 
Municipal population census (December 2020) - Information about the population of each neighborhood, including gender, nationality and age.

We will use maps to represent some of the information. For that, we will need geographical data about Madrid. We will use the following resources:

-Madrid Neighborhoods (geojson)
-Madrid districts (geojson)
Before getting into the capture and analysis of the data, let's import some common use libraries:

<H3>2.1. Madrid Neighborhoods</h3>
The last cell), the City of Madrid has 131 neighborhoods. The Municipality of Madrid provides information about every neighborhood in CSV format in this link: https://datos.madrid.es/egob/catalogo/200078-1-distritos-barrios.csv. Let's start by getting the data

In [2]:
#getting data from barrios of Madrid
url='data/Barrios1.csv'

In [3]:
# Then we read the csv file we have downloaded into a pandas dataframe
madrid = pd.read_csv(url, sep=';',encoding='ISO-8859-1')

In [4]:
madrid.dtypes

OBJECTID_1     int64
Districto     object
Barrio        object
Shape_Leng    object
Shape_Area    object
COD_DIS        int64
COD_DIS_TX     int64
BARRIO_MAY    object
COD_DISBAR     int64
COD_BAR        int64
NUM_BAR        int64
BARRIO_MT     object
COD_DISB      object
dtype: object

In [5]:
madrid.head()

,OBJECTID_1,Districto,Barrio,Shape_Leng,Shape_Area,COD_DIS,COD_DIS_TX,BARRIO_MAY,COD_DISBAR,COD_BAR,NUM_BAR,BARRIO_MT,COD_DISB
0,60,Centro,Palacio,"5754,822748","1469905,684",1,1,PALACIO,11,101,1,PALACIO,1_1
1,50,Centro,Embajadores,"4275,227681","1033724,698",1,1,EMBAJADORES,12,102,2,EMBAJADORES,1_2
2,55,Centro,Cortes,"3731,07903","591874,1219",1,1,CORTES,13,103,3,CORTES,1_3
3,64,Centro,Justicia,"3597,421427","739414,338",1,1,JUSTICIA,14,104,4,JUSTICIA,1_4
4,66,Centro,Universidad,"4060,075813","948027,0773",1,1,UNIVERSIDAD,15,105,5,UNIVERSIDAD,1_5


In [6]:
madrid.drop (['OBJECTID_1','COD_DIS_TX','BARRIO_MAY','BARRIO_MT'], axis='columns', inplace=True)

In [7]:
madrid.head()

,Districto,Barrio,Shape_Leng,Shape_Area,COD_DIS,COD_DISBAR,COD_BAR,NUM_BAR,COD_DISB
0,Centro,Palacio,"5754,822748","1469905,684",1,11,101,1,1_1
1,Centro,Embajadores,"4275,227681","1033724,698",1,12,102,2,1_2
2,Centro,Cortes,"3731,07903","591874,1219",1,13,103,3,1_3
3,Centro,Justicia,"3597,421427","739414,338",1,14,104,4,1_4
4,Centro,Universidad,"4060,075813","948027,0773",1,15,105,5,1_5


In [12]:
madrid.rename(columns={'Barrio':'neighborhood','COD_BAR':'code','COD_DIS':'district_id','COD_DISBAR':'distNeigh_Id','NUM_BAR':'neighborhood_id','COD_DISB':'Cod_distNeigh'} ,inplace=True)
madrid.head()

,Districto,neighborhood,Shape_Leng,Shape_Area,district_id,distNeigh_Id,code,neighborhood_id,Cod_distNeigh
0,Centro,Palacio,5754.822748,1469905.684,1,11,101,1,1_1
1,Centro,Embajadores,4275.227681,1033724.698,1,12,102,2,1_2
2,Centro,Cortes,3731.07903,591874.1219,1,13,103,3,1_3
3,Centro,Justicia,3597.421427,739414.338,1,14,104,4,1_4
4,Centro,Universidad,4060.075813,948027.0773,1,15,105,5,1_5


In [10]:
# replace ',' by '.'
madrid['Shape_Area']=[x.replace(",",".") for x in madrid['Shape_Area']]
madrid['Shape_Leng']=[x.replace(",",".") for x in madrid['Shape_Leng']]

In [13]:
madrid.dtypes

Districto          object
neighborhood       object
Shape_Leng         object
Shape_Area         object
district_id         int64
distNeigh_Id        int64
code                int64
neighborhood_id     int64
Cod_distNeigh      object
dtype: object

The neighborhoods table has the following columns:
<UL>
<LI type="square">Districto:Name od the district.</li>
<LI type="square">neighborhood:Name of the neighborhood </li>
<LI type="square">Shape_Leng:Perimeter</li>
<LI type="square">Shape_Area:Area</li>
<LI type="square">district_id :Unique code that identifies the district the neighborhood belongs to. This code is the same as the district code from the district table.</li>
<LI type="square">distNeigh_Id:Unique code that concatenates the district code and the neighborhood code.</li>
<LI type="square">Code:Unique code that concatenates the district code and the neighborhood code. Unlike the previous field, the neighborhood is taken in two digits. For example, district 1 and neighborhood 1 would become code = 101</li>
<LI type="square">neighborhood_id:unique code that identifies  the neighborhood belongs to. </li>
<LI type="square">Cod_distNeigh:Concatenation district with neighborhood</li>
 </ul>
This dataframe does not include relevant geographic information.


In [15]:
# We will center all city maps around this coordinate
mad_ll=[40.46206,-3.6766]

In [16]:
# We read the csv file containing the coordinates for each neighborhood, check the first rows
url2='data/barrios_geo1.csv'

madrid_coor = pd.read_csv(url2, sep=';',encoding='ISO-8859-1')

In [17]:
madrid_coor.head()

,code,latitude,longitude
0,101,40.416841,-3.711605
1,201,40.406598,-3.719600
2,301,40.404606,-3.678362
3,401,40.424700,-3.685462
4,501,40.449060,-3.686285


In [18]:
madrid_coor.dtypes

code           int64
latitude     float64
longitude    float64
dtype: object

In [19]:
# As we have the code, we merge the dataframe with information about the neighborhood and
# the dataframe with the coordinates for each neighborhood. Check the resulting dataframe

madrid_coor['code'] = madrid_coor['code'].astype('str')
madrid['code'] = madrid['code'].astype('str')
madrid = madrid.merge(madrid_coor, on="code")



In order to get an idea of the distribution of the neighborhoods - and also to check the accuracy of the coordinates we got from the geocoding service, let's print a map that includes a marker in each neighborhood coordinate, as well as the neighborhood limits obtained from a .geojson file available in CartoDB (link here)

In [20]:
madrid.dtypes

Districto           object
neighborhood        object
Shape_Leng          object
Shape_Area          object
district_id          int64
distNeigh_Id         int64
code                object
neighborhood_id      int64
Cod_distNeigh       object
latitude           float64
longitude          float64
dtype: object

In [22]:
neighborhoods_geojson = f"data/barrios.geojson"

mn = folium.Map(location=mad_ll, zoom_start=11)

folium.GeoJson(neighborhoods_geojson, name="geojson").add_to(mn)

for name, code, lat, lng in list(zip(madrid['neighborhood'], madrid['code'], madrid['latitude'], madrid['longitude'])):
    folium.Circle(
        radius=50,
        location=[lat,lng],
        popup="{} - {}".format(code, name),
        color="black",
        fill=True,
        fill_color="black"
    ).add_to(mn)

mn



<H3>2.2. Venues and their activities (from the Open Data Portal)</H3>
In the following sections we will use the FourSquare API to get information about the recommended venues in the neighborhoods of Madrid. However, the Municipality of Madrid provides a catalog of all venues and their activities in the city of Madrid, updated monthly. Let's take a look at the information available here.

In [24]:

url3='data/Locales202012.csv'
# We have already downloaded the CSV file, so let's read it using read_csv
venues = pd.read_csv(url3,
                     sep=";",
                     header=0,
                     index_col=0,
                     usecols=list(range(40)),
                     encoding="ISO-8859-1")

In [25]:
# Let's take a look at the first 5 rows to get an idea of the information available
venues.head()

,id_distrito_local,desc_distrito_local,id_barrio_local,desc_barrio_local,cod_barrio_local,id_seccion_censal_local,desc_seccion_censal_local,coordenada_x_local,coordenada_y_local,id_tipo_acceso_local,...,cal_acceso,coordenada_x_agrupacion,coordenada_y_agrup,id_agrupacion,nombre_agrupacion,id_tipo_agrup,desc_tipo_agrup,id_planta_agrupado,id_local_agrupado,rotulo
id_local,,,,,,,,,,,,,,,,,,,,,
10000004,1,CENTRO,105,UNIVERSIDAD,NaN,NaN,106,"439945,6","4475591,53",NaN,...,,NaN,NaN,NaN,NaN,NaN,NaN,PB,NaN,BAR LA PARADA ...
10000105,1,CENTRO,104,JUSTICIA,NaN,NaN,84,"440958,59","4475092,52",NaN,...,,NaN,NaN,NaN,NaN,NaN,NaN,PB,NaN,LA MORDIDA
10000162,1,CENTRO,104,JUSTICIA,NaN,NaN,88,0,0,NaN,...,,"440692,59","4475369,53",99000056.0,MERCADO MUNICIPAL DE BARCELO,12.0,Mercado Municipal,0,101,POLLERIAS RODRIGUEZ & MARTINEZ
10000224,1,CENTRO,102,EMBAJADORES,NaN,NaN,35,0,0,NaN,...,,"440295,59","4473246,53",99000057.0,MERCADO MUNICIPAL DE SAN FERNANDO,12.0,Mercado Municipal,PB,70,SIN ROTULO
10000284,1,CENTRO,103,CORTES,NaN,NaN,65,"440755,59","4473926,52",NaN,...,,NaN,NaN,NaN,NaN,NaN,NaN,PB,NaN,RESTAURANTE DAR MONA


In [26]:
# let's explore the structure a little deeper using .info()
venues.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 146514 entries, 10000004 to 290001652
Data columns (total 39 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   id_distrito_local          146514 non-null  int64  
 1   desc_distrito_local        146514 non-null  object 
 2   id_barrio_local            146514 non-null  int64  
 3   desc_barrio_local          146514 non-null  object 
 4   cod_barrio_local           0 non-null       float64
 5   id_seccion_censal_local    0 non-null       float64
 6   desc_seccion_censal_local  146514 non-null  int64  
 7   coordenada_x_local         146514 non-null  object 
 8   coordenada_y_local         146514 non-null  object 
 9   id_tipo_acceso_local       0 non-null       float64
 10  desc_tipo_acceso_local     146514 non-null  object 
 11  id_situacion_local         146514 non-null  int64  
 12  desc_situacion_local       146514 non-null  object 
 13  id_vial_edificio   

In [27]:
venues_final = venues[['id_distrito_local', 'desc_distrito_local',
                       'id_barrio_local', 'desc_barrio_local',
                       'coordenada_x_local', 'coordenada_y_local',
                       'id_situacion_local', 'desc_situacion_local',
                       'rotulo']]
venues_final.head()

,id_distrito_local,desc_distrito_local,id_barrio_local,desc_barrio_local,coordenada_x_local,coordenada_y_local,id_situacion_local,desc_situacion_local,rotulo
id_local,,,,,,,,,
10000004,1,CENTRO,105,UNIVERSIDAD,"439945,6","4475591,53",1,Abierto,BAR LA PARADA ...
10000105,1,CENTRO,104,JUSTICIA,"440958,59","4475092,52",1,Abierto,LA MORDIDA
10000162,1,CENTRO,104,JUSTICIA,0,0,1,Abierto,POLLERIAS RODRIGUEZ & MARTINEZ
10000224,1,CENTRO,102,EMBAJADORES,0,0,1,Abierto,SIN ROTULO
10000284,1,CENTRO,103,CORTES,"440755,59","4473926,52",1,Abierto,RESTAURANTE DAR MONA


In [28]:
venues_final_en = venues_final.rename(columns={'id_distrito_local': 'district_id',
                                            'desc_distrito_local': 'district_name',
                                            'id_barrio_local': 'neighborhood_id',
                                            'desc_barrio_local': 'neighborhood_name',
                                            'coordenada_x_local': 'x_coordinate',
                                            'coordenada_y_local': 'y_coordinate',
                                            'id_situation_local': 'status_id',
                                            'desc_situacion_local': 'status',
                                            'rotulo': 'name'})

The status code has a complicated definition; although there are several procedures for updating data about activities, there is no code that provides information when an activity ends and the venue closes without any new activities. The following table presents a description of the different values for the situation:

Code/Type of situation	Description of situation
1 Open	Active venue
2 Closed	Closed venue (no activity)
7 Works	Venue at work
8 Extinct	Extinct venue
9 Extinct by Reunification	Venue that has joined another venue (and disappeared)
5 Use for accommodation	Venue for living

In [23]:
venues_final_en["status"].value_counts()

Abierto               100965
Cerrado                33337
Uso vivienda            6346
Baja Reunificacion      2933
Baja                    2159
En obras                 767
Baja PC Asociado           7
Name: status, dtype: int64

This dataset reports 146,514 venues in Madrid in December 2020: 100,965 open, 33,337 closed and 12,212 in other situations. We are interested in extracting this information, so let's create a 'dummy' dataframe, where this values are converted into columns. We use the method pandas.get_dummies()

In [24]:
dummy = pd.get_dummies(venues_final_en['status'])
dummy.head()

,Abierto,Baja,Baja PC Asociado,Baja Reunificacion,Cerrado,En obras,Uso vivienda
id_local,,,,,,,
10000004,1,0,0,0,0,0,0
10000105,1,0,0,0,0,0,0
10000162,1,0,0,0,0,0,0
10000224,1,0,0,0,0,0,0
10000284,1,0,0,0,0,0,0


In [25]:
# Let's translate the columns for easeness
dummy_en = dummy.rename(columns={'Abierto': 'open',
                                 'Baja': 'unregistered',
                                 'Baja PC Asociado': 'unregistered PC',
                                 'Baja Reunificacion': 'reunified',
                                 'Cerrado': 'closed',
                                 'En obras': 'under construction',
                                 'Uso vivienda': 'for accommodation'})
dummy_en.columns

Index(['open', 'unregistered', 'unregistered PC', 'reunified', 'closed',
       'under construction', 'for accommodation'],
      dtype='object')

In [26]:
# Finally, let's concat both dataframes, so we get the status information as a column
venues_status = pd.concat([venues_final_en, dummy_en], axis=1)

In [27]:
# We are interested in the number of venues per status per neighborhood. 
# We group rows by neighborhood and sum all columns
venues_status = venues_status.groupby(['neighborhood_id', 'neighborhood_name']).sum()[list(dummy_en.columns)]
venues_status.sample(5)

,,open,unregistered,unregistered PC,reunified,closed,under construction,for accommodation
neighborhood_id,neighborhood_name,,,,,,,
1306,NUMANCIA,1249.0,9.0,0.0,50.0,690.0,6.0,242.0
402,GOYA,1560.0,1.0,0.0,33.0,103.0,2.0,2.0
1204,ALMENDRALES,652.0,0.0,0.0,8.0,331.0,3.0,43.0
806,VALVERDE,1344.0,2.0,0.0,41.0,937.0,2.0,108.0
2101,ALAMEDA DE OSUNA,318.0,2.0,0.0,3.0,117.0,0.0,4.0


In [28]:
# Finally, let's create a column that aggregates the total number of venues per neighborhood
venues_status['total'] = venues_status.sum(axis=1)
venues_status.sample(3)

,,open,unregistered,unregistered PC,reunified,closed,under construction,for accommodation,total
neighborhood_id,neighborhood_name,,,,,,,,
401,RECOLETOS,1430.0,3.0,0.0,36.0,56.0,14.0,0.0,1539.0
504,HISPANOAMERICA,1269.0,2.0,0.0,18.0,142.0,4.0,22.0,1457.0
704,ALMAGRO,983.0,0.0,0.0,31.0,103.0,3.0,11.0,1131.0


In [29]:
# Maybe it will be interesting at some point to compare the 'density' of venues instead of the
# total number of venues (it makes sense that bigger neighborhoods would have more venues).
# Fortunately, we have information about the ares of each neighborhood in our nb_final dataframe

venues_status['area'] = [madrid.loc[madrid['code']== str(code), 'Shape_Area'].iloc[0] for neighborhood_id in venues_status.index.get_level_values(0)]
venues_status.head()

,,open,unregistered,unregistered PC,reunified,closed,under construction,for accommodation,total,area
neighborhood_id,neighborhood_name,,,,,,,,,
101,PALACIO,1554.0,13.0,0.0,38.0,345.0,15.0,57.0,2022.0,4682537.436
102,EMBAJADORES,2724.0,30.0,0.0,101.0,628.0,31.0,111.0,3625.0,4682537.436
103,CORTES,1062.0,11.0,0.0,18.0,127.0,5.0,9.0,1232.0,4682537.436
104,JUSTICIA,1750.0,4.0,0.0,40.0,105.0,4.0,5.0,1908.0,4682537.436
105,UNIVERSIDAD,2149.0,19.0,0.0,84.0,294.0,51.0,96.0,2693.0,4682537.436


In [52]:

# The geojson file that we will use for Chorophleth maps identifies the neighborhoods by
# concatenating the district code + a hyphen (in lieu of the zneighborhoods_geojsonero) + the neighborhood code.
# This function replaces the zeros in the code with hyphens in case we need to repeat this action 
def replaceZeroWithHyphen(codes):
    return [code[:-2] + "-" + code[-1] for code in codes.astype(str)]

In [31]:

# Finally, in order to visualize the data in a Choropleth map in Folium, we need to update our 'code' field
# in order to make compatible with the code information in the geojson file. In the geojson file, the code
# information has the format [district] + "-" + [neighborhood], while we are using a format
# [district] + "0" + [neighborhood].  Hence, we need to create a new column named 'codbarrio' and to replace
# the middle "0" with a "-"
venues_status['codbarrio'] = replaceZeroWithHyphen(venues_status.index.get_level_values(0))

In [37]:

venues_status.head(2)


,,open,unregistered,unregistered PC,reunified,closed,under construction,for accommodation,total,area,codbarrio
neighborhood_id,neighborhood_name,,,,,,,,,,
101,PALACIO,1554.0,13.0,0.0,38.0,345.0,15.0,57.0,2022.0,4682537.436,1-1
102,EMBAJADORES,2724.0,30.0,0.0,101.0,628.0,31.0,111.0,3625.0,4682537.436,1-2


In [39]:
# Let0s check the map with the total venues
bins = list(venues_status['total'].quantile([0, 0.4, 0.75, 0.9, 0.97, 1]))

m = folium.Map(location=mad_ll, zoom_start=11)
  

for name, lat, lng in list(zip(madrid['Barrio'], madrid['latitude'], madrid['longitude'])):
    folium.Circle(
        radius=100,
        location=[lat,lng],
        popup=name,
        color="black",
        fill=True,
        fill_color="black"
    ).add_to(m)

m

In [40]:
# In order to check the venues density, let's create a new 'density' column that we wil get
# by dividing the total number per area. But first, let's check if we have any null area values
venues_status[venues_status['area'] == 0]

,,open,unregistered,unregistered PC,reunified,closed,under construction,for accommodation,total,area,codbarrio
neighborhood_id,neighborhood_name,,,,,,,,,,


<H2>2.4. Venues per Neighborhood (from FourSquare)</H2>

I initialize the necessary variables to make the call with the data from my Foursquare account

In [37]:

CLIENT_ID = '35ARGH5HI3QICIBSYDMQSJ2LFTYPF0HLBKR3AOT150VPFUGL' # your Foursquare ID
CLIENT_SECRET = 'YZVTIIVC0HYCERFWBQ4G0LRGMLS5JV2IUJDGSJ2LMJWC5D5U' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value
radius = 500
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 35ARGH5HI3QICIBSYDMQSJ2LFTYPF0HLBKR3AOT150VPFUGL
CLIENT_SECRET:YZVTIIVC0HYCERFWBQ4G0LRGMLS5JV2IUJDGSJ2LMJWC5D5U


In [35]:
neighborhood_latitude = madrid.loc[0, 'latitude'] # neighborhood latitude value
neighborhood_longitude = madrid.loc[0, 'longitude'] # neighborhood longitude value


Creating url from my credentials

In [39]:
urlAPI = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

urlAPI

'https://api.foursquare.com/v2/venues/explore?&client_id=35ARGH5HI3QICIBSYDMQSJ2LFTYPF0HLBKR3AOT150VPFUGL&client_secret=YZVTIIVC0HYCERFWBQ4G0LRGMLS5JV2IUJDGSJ2LMJWC5D5U&v=20180605&ll=40.4168407596632,-3.7116049269648497&radius=500&limit=100'

This function will get the venues in a radius of 500 m given the latitude and longitude values or each neighborhood

In [40]:
def getNearbyVenues(codes, names, latitudes, longitudes):
    venues_list = []
    
    for code, name, lat, lng in list(zip(codes, names, latitudes, longitudes)):
        

        # Make the get request
        try:
            results = requests.get(urlAPI).json()["response"]["groups"][0]["items"]
        
            # return only relevant information for each nearby venue
            venues_list.append([(
                code,
                name,
                v['venue']['name'],
                v['venue']['location']['lat'],
                v['venue']['location']['lng'],
                v['venue']['categories'][0]['name']
            ) for v in results])
        except:
            pass
        
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = [
        'Neighborhood code',
        'Neighborhood name',
        'Venue',
        'Venue Latitude',
        'Venue Longitude',
        'Venue Category'
    ]
    return nearby_venues

accessing Foursquare

In [42]:
venues = getNearbyVenues(codes=madrid['code'],
                           names=madrid['neighborhood'],
                            latitudes=madrid['latitude'],
                            longitudes=madrid['longitude'])

In [77]:
venues = pd.read_csv('data/venues.csv', index_col=0)

In [80]:
venues.head()

,Neighborhood code,Neighborhood name,Venue,Venue Latitude,Venue Longitude,Venue Category
0,101,PALACIO,la gastroteca de santiago,40.416639,-3.710944,Restaurant
1,101,PALACIO,La Esquina del Real,40.417356,-3.710364,French Restaurant
2,101,PALACIO,Gayane's Madrid,40.416968,-3.709632,Caucasian Restaurant
3,101,PALACIO,Gyoza Go!,40.416179,-3.708612,Dumpling Restaurant
4,101,PALACIO,jamones julian becerro,40.417867,-3.708744,Deli / Bodega


In [78]:
venues.shape

(11482, 6)

In [79]:
venues['Venue Category'].value_counts()

Spanish Restaurant         2182
Restaurant                 1313
Tapas Restaurant            878
Café                        625
Pizza Place                 615
                           ... 
Grilled Meat Restaurant       2
Bagel Shop                    2
Szechuan Restaurant           1
Gluten-free Restaurant        1
Pet Café                      1
Name: Venue Category, Length: 96, dtype: int64

Our 11482 stores are divided into several categories, the most popular being 'Spanish Restaurant', 'Restaurant' and 'Tapas Restaurant', all of them typical local restaurants.

In [81]:
venues.groupby('Neighborhood code').count()['Venue']

Neighborhood code
101      72
102      82
103     100
104     100
105     100
       ... 
2101    100
2102    100
2103     92
2104     77
2105     95
Name: Venue, Length: 131, dtype: int64

In [82]:
# let's add the 'codbarrio' column for using in a possible Choropleth map
venues['codbarrio'] = replaceZeroWithHyphen(venues['Neighborhood code'].astype(str))

In [83]:
venues.head()

,Neighborhood code,Neighborhood name,Venue,Venue Latitude,Venue Longitude,Venue Category,codbarrio
0,101,PALACIO,la gastroteca de santiago,40.416639,-3.710944,Restaurant,1-1
1,101,PALACIO,La Esquina del Real,40.417356,-3.710364,French Restaurant,1-1
2,101,PALACIO,Gayane's Madrid,40.416968,-3.709632,Caucasian Restaurant,1-1
3,101,PALACIO,Gyoza Go!,40.416179,-3.708612,Dumpling Restaurant,1-1
4,101,PALACIO,jamones julian becerro,40.417867,-3.708744,Deli / Bodega,1-1


<H2>2.5. Venues by price category</H2>
The explore endpoint of the Foursquare API includes a price parameter, which allows developers to explore venues around a location based on their price range.

In this section, we will explore the recommended venues per price range in each neighborhood. With this, we will be able to explore the areas where the most expensive restaurants are located, the areas with the higher ratio of



In [56]:
# Url with price
def build_url_with_price(lat, lng, price):
    url = 'https://api.foursquare.com/v2/venues/explore?' +'&client_id={}'.format(CLIENT_ID) +'&client_secret={}&v={}'.format(CLIENT_SECRET, VERSION) + '&section=food&limit={}'.format(LIMIT) + '&ll={},{}'.format(lat,lng) + '&price={}'.format(price)
    return url

In [59]:
def getNearbyVenuesByPrice(codes, names, latitudes, longitudes):
    venues_list = []
    
    for code, name, lat, lng in list(zip(codes, names, latitudes, longitudes)):
        
        for price in range(1, 5):
        
            # Create the API request URL using the build_url function
            url = build_url_with_price(lat, lng, price)
        
            # Make the get request
            try:
                results = requests.get(url).json()["response"]["groups"][0]["items"]
        
                # return only relevant information for each nearby venue
                venues_list.append([(
                    code,
                    name,
                    price,
                    v['venue']['name'],
                    v['venue']['location']['lat'],
                    v['venue']['location']['lng'],
                    v['venue']['categories'][0]['name']
                ) for v in results])
            except:
                pass
        
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = [
        'Neighborhood code',
        'Neighborhood name',
        'Price',
        'Venue',
        'Venue Latitude',
        'Venue Longitude',
        'Venue Category'
    ]
    return nearby_venues

In [60]:
# Here we use the function we created in the previous cell to call the Foursquare API and 
# store the results in a dataframe 
venues_price = getNearbyVenuesByPrice(codes=madrid['code'],
                                      names=madrid['neighborhood'],
                                      latitudes=madrid['latitude'],
                                      longitudes=madrid['longitude'])

In [61]:
venues_price.head()

,Neighborhood code,Neighborhood name,Price,Venue,Venue Latitude,Venue Longitude,Venue Category
0,101,Palacio,1,Gyoza Go!,40.416179,-3.708612,Dumpling Restaurant
1,101,Palacio,1,Mi Ciudad I,40.417478,-3.708625,Mexican Restaurant
2,101,Palacio,1,jamones julian becerro,40.417867,-3.708744,Deli / Bodega
3,101,Palacio,1,Federal Café 2,40.414460,-3.708774,Café
4,101,Palacio,1,100 Montaditos,40.418571,-3.710714,Sandwich Place


In [62]:
venues_price.tail()

,Neighborhood code,Neighborhood name,Price,Venue,Venue Latitude,Venue Longitude,Venue Category
12506,2105,Corralejos,3,Cañas y Tapas,40.447750,-3.587187,Tapas Restaurant
12507,2105,Corralejos,3,Al Tran Tran,40.472403,-3.582019,Tapas Restaurant
12508,2105,Corralejos,3,La Consentida,40.472353,-3.582128,Gastropub
12509,2105,Corralejos,3,El Chiringo,40.454185,-3.600374,Tapas Restaurant
12510,2105,Corralejos,3,Balcón de Las Naciones,40.461948,-3.621289,Paella Restaurant


In [63]:
venues_price.shape

(12511, 7)

<H2>3. Analysis</H2>
<H3>3.1. Clustering by venue category</H3>
First, we would like to use machine learning techniques to cluster neighborhoods depending on the category of their venues. In order to do that, we would need to build a dataframe with the total number of venues per category type per neighborhood.
We are interested in getting the number of venues per category per neighborhood. 
We create a 'dummies' dataframe with the categories and append it to our venues dataframe

In [84]:
categories = pd.get_dummies(venues['Venue Category'])
venues_wdummies = pd.concat([venues, categories], axis=1)
venues_wdummies.head()

,Neighborhood code,Neighborhood name,Venue,Venue Latitude,Venue Longitude,Venue Category,codbarrio,American Restaurant,Arepa Restaurant,Argentinian Restaurant,...,Taco Place,Tapas Restaurant,Taverna,Thai Restaurant,Theme Restaurant,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Vietnamese Restaurant
0,101,PALACIO,la gastroteca de santiago,40.416639,-3.710944,Restaurant,1-1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,101,PALACIO,La Esquina del Real,40.417356,-3.710364,French Restaurant,1-1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,101,PALACIO,Gayane's Madrid,40.416968,-3.709632,Caucasian Restaurant,1-1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,101,PALACIO,Gyoza Go!,40.416179,-3.708612,Dumpling Restaurant,1-1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,101,PALACIO,jamones julian becerro,40.417867,-3.708744,Deli / Bodega,1-1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


We group by neighborhood and get the total number of restaurants per category per neighborhood

In [85]:
venues_categories_sum = venues_wdummies.groupby(['Neighborhood code', 'Neighborhood name']).sum()[categories.columns]
venues_categories_sum.head()

,,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bistro,Brazilian Restaurant,Breakfast Spot,...,Taco Place,Tapas Restaurant,Taverna,Thai Restaurant,Theme Restaurant,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Vietnamese Restaurant
Neighborhood code,Neighborhood name,,,,,,,,,,,,,,,,,,,,,
101,PALACIO,1,0,1,0,0,0,0,2,0,1,...,0,19,0,0,0,0,0,2,0,0
102,EMBAJADORES,1,0,3,1,0,0,2,2,0,3,...,0,9,0,0,0,0,0,4,1,1
103,CORTES,0,0,2,1,1,0,1,1,0,2,...,0,14,1,1,0,0,0,1,1,0
104,JUSTICIA,2,0,0,1,1,0,6,4,0,0,...,0,4,0,2,0,0,0,3,0,0
105,UNIVERSIDAD,1,0,4,2,0,0,4,0,0,3,...,3,9,0,0,0,0,0,5,0,1


In [86]:
venues_categories_sum.tail()

,,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bistro,Brazilian Restaurant,Breakfast Spot,...,Taco Place,Tapas Restaurant,Taverna,Thai Restaurant,Theme Restaurant,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Vietnamese Restaurant
Neighborhood code,Neighborhood name,,,,,,,,,,,,,,,,,,,,,
2101,ALAMEDA DE OSUNA,1,0,2,4,0,0,3,2,0,2,...,0,4,0,0,0,0,0,0,0,0
2102,AEROPUERTO,2,0,2,2,1,0,3,2,0,2,...,0,4,0,1,0,1,0,0,0,0
2103,CASCO H.BARAJAS,1,0,2,0,1,0,1,2,0,5,...,0,5,0,1,0,0,0,0,0,0
2104,TIMON,0,0,2,4,0,0,2,2,0,2,...,0,6,0,1,0,1,0,0,0,0
2105,CORRALEJOS,1,0,2,1,0,0,2,2,0,2,...,0,4,0,1,0,1,0,0,0,0


In order to use the K-Means algorithm, we will standardize the values of the dataframe so they are between 0 and 1. We will use the Normalizar preprocessor.

In [87]:
se = Normalizer()
venues_categories_std = se.fit_transform(venues_categories_sum)
venues_categories_std

array([[0.03898406, 0.        , 0.03898406, ..., 0.07796812, 0.        ,
        0.        ],
       [0.04526787, 0.        , 0.13580362, ..., 0.18107149, 0.04526787,
        0.04526787],
       [0.        , 0.        , 0.06644555, ..., 0.03322277, 0.03322277,
        0.        ],
       ...,
       [0.03747658, 0.        , 0.07495317, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.08679261, ..., 0.        , 0.        ,
        0.        ],
       [0.03771571, 0.        , 0.07543143, ..., 0.        , 0.        ,
        0.        ]])

KMeans to Cluster. K=5

In [88]:
# Then use KMeans to cluster to divide the values into 5 clusters
kmeans = KMeans(n_clusters=5, random_state=0).fit(venues_categories_std)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([1, 1, 1, 2, 1, 1, 1, 1, 0, 2, 1, 1, 1, 0, 2, 1, 1, 2, 0, 2, 0, 0,
       0, 2, 2, 2, 1, 1, 2, 2, 2, 0, 1, 0, 2, 0, 0, 1, 1, 1, 2, 1, 0, 2,
       1, 3, 1, 1, 2, 4, 2, 2, 1, 2, 0, 2, 2, 2, 3, 1, 4, 4, 4, 4, 4, 0,
       3, 4, 4, 4, 4, 4, 4, 3, 3, 0, 0, 0, 0, 3, 2, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 0, 0, 0, 0, 2, 4, 3, 3, 0,
       4, 3, 3, 4, 3, 3, 3, 3, 2, 2, 2, 3, 2, 3, 2, 2, 2, 2, 2, 2, 2])

In [89]:

# We then add the calculated label to the neighborhood
venues_categories_sum['label'] = kmeans.labels_
venues_categories_sum.head(3)

,,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bistro,Brazilian Restaurant,Breakfast Spot,...,Tapas Restaurant,Taverna,Thai Restaurant,Theme Restaurant,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Vietnamese Restaurant,label
Neighborhood code,Neighborhood name,,,,,,,,,,,,,,,,,,,,,
101,PALACIO,1,0,1,0,0,0,0,2,0,1,...,19,0,0,0,0,0,2,0,0,1
102,EMBAJADORES,1,0,3,1,0,0,2,2,0,3,...,9,0,0,0,0,0,4,1,1,1
103,CORTES,0,0,2,1,1,0,1,1,0,2,...,14,1,1,0,0,0,1,1,0,1


In [90]:
# In order to get an idea of the characteristics of the different labels, we sum 
venues_categories_per_label = venues_categories_sum.groupby('label').sum()
venues_categories_per_label.head(5)

,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bistro,Brazilian Restaurant,Breakfast Spot,...,Taco Place,Tapas Restaurant,Taverna,Thai Restaurant,Theme Restaurant,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Vietnamese Restaurant
label,,,,,,,,,,,,,,,,,,,,,
0,17.0,2.0,34.0,64.0,28.0,0.0,133.0,10.0,13.0,46.0,...,1.0,203.0,0.0,11.0,2.0,1.0,0.0,3.0,3.0,4.0
1,13.0,4.0,23.0,30.0,14.0,0.0,72.0,15.0,7.0,41.0,...,10.0,253.0,1.0,12.0,5.0,0.0,2.0,39.0,2.0,3.0
2,31.0,2.0,40.0,110.0,18.0,2.0,116.0,39.0,2.0,75.0,...,0.0,179.0,1.0,20.0,4.0,6.0,0.0,8.0,2.0,0.0
3,30.0,0.0,7.0,45.0,15.0,0.0,92.0,4.0,8.0,41.0,...,0.0,146.0,0.0,1.0,0.0,0.0,5.0,2.0,0.0,0.0
4,20.0,0.0,1.0,41.0,22.0,0.0,57.0,0.0,17.0,21.0,...,9.0,97.0,0.0,0.0,0.0,4.0,3.0,1.0,2.0,0.0


<h3>3.2. Clustering by venue price</h3>

We have the information about the price in a dataframe column. However, in order to apply any machine learning algorithms we need to have this information in their own column. We will use the get_dummies method in order to get a dataframe where the columns will be the price category, and the value will be a boolean-like integer.

In [91]:
venues_price.head()

,Neighborhood code,Neighborhood name,Price,Venue,Venue Latitude,Venue Longitude,Venue Category
0,101,Palacio,1,Gyoza Go!,40.416179,-3.708612,Dumpling Restaurant
1,101,Palacio,1,Mi Ciudad I,40.417478,-3.708625,Mexican Restaurant
2,101,Palacio,1,jamones julian becerro,40.417867,-3.708744,Deli / Bodega
3,101,Palacio,1,Federal Café 2,40.414460,-3.708774,Café
4,101,Palacio,1,100 Montaditos,40.418571,-3.710714,Sandwich Place


Let's move the price information into columns


In [92]:
price_dummies = pd.get_dummies(venues_price['Price'])
price_dummies.rename(columns={1: 'Price 1', 2: 'Price 2', 3: 'Price 3', 4: 'Price 4'}, inplace=True)
price_dummies.head()

,Price 1,Price 2,Price 3,Price 4
0,1,0,0,0
1,1,0,0,0
2,1,0,0,0
3,1,0,0,0
4,1,0,0,0


In [93]:
# Then, we concat the venues dataframe with the prices dataframe
venues_price_cat = pd.concat([venues_price, price_dummies], axis=1)
venues_price_cat.head()

,Neighborhood code,Neighborhood name,Price,Venue,Venue Latitude,Venue Longitude,Venue Category,Price 1,Price 2,Price 3,Price 4
0,101,Palacio,1,Gyoza Go!,40.416179,-3.708612,Dumpling Restaurant,1,0,0,0
1,101,Palacio,1,Mi Ciudad I,40.417478,-3.708625,Mexican Restaurant,1,0,0,0
2,101,Palacio,1,jamones julian becerro,40.417867,-3.708744,Deli / Bodega,1,0,0,0
3,101,Palacio,1,Federal Café 2,40.414460,-3.708774,Café,1,0,0,0
4,101,Palacio,1,100 Montaditos,40.418571,-3.710714,Sandwich Place,1,0,0,0


In [94]:

# We then group the price categories per neighborhood
prices_df = venues_price_cat.groupby(['Neighborhood code', 'Neighborhood name']).sum()[['Price 1', 'Price 2', 'Price 3', 'Price 4']]
prices_df

,,Price 1,Price 2,Price 3,Price 4
Neighborhood code,Neighborhood name,,,,
1001,CArmenes,36,35,6,1
1002,Puerta del Angel,50,73,26,1
1003,Lucero,43,39,9,1
1004,Aluche,24,20,14,0
1005,Campamento,32,42,16,1
...,...,...,...,...,...
903,Ciudad Universitaria,19,22,5,0
904,Valdezarza,31,29,6,1
905,ValdemarIn,29,63,10,2


In order to get an idea of the number of venues per price category, we sum the columns


In [95]:
prices_df.sum(axis=0)

Price 1    4553
Price 2    6348
Price 3    1533
Price 4      77
dtype: int64

We can see the most common type of restaurant is the restaurant tagged with 'Price 2'. There are not many expensive restaurants in Madrid.
We add this information to our neighborhoods dataframe, in order to get geographical information

In [101]:
madrid_prices = madrid.merge(prices_df, left_on='code', right_on='Neighborhood code')
madrid_prices.head()

,Districto,neighborhood,Shape_Leng,Shape_Area,district_id,distNeigh_Id,code,neighborhood_id,Cod_distNeigh,latitude,longitude,Price 1,Price 2,Price 3,Price 4
0,Centro,Palacio,5754.822748,1469905.684,1,11,101,1,1_1,40.416841,-3.711605,13,32,25,0
1,Centro,Embajadores,4275.227681,1033724.698,1,12,102,2,1_2,40.409836,-3.702715,21,47,7,0
2,Centro,Cortes,3731.07903,591874.1219,1,13,103,3,1_3,40.414867,-3.697085,26,81,21,0
3,Centro,Justicia,3597.421427,739414.338,1,14,104,4,1_4,40.423301,-3.696126,22,74,9,1
4,Centro,Universidad,4060.075813,948027.0773,1,15,105,5,1_5,40.426313,-3.706528,37,70,16,0


We will cluster the neighborhoods per price category, so let's extract that information


In [102]:
prices =madrid_prices[['Price 1', 'Price 2', 'Price 3', 'Price 4']]
prices.head()

,Price 1,Price 2,Price 3,Price 4
0,13,32,25,0
1,21,47,7,0
2,26,81,21,0
3,22,74,9,1
4,37,70,16,0


We use standard scaler to get ready the document for processing


In [99]:
scaler = StandardScaler()
prices_std = scaler.fit_transform(prices)

K MEans . K=4

In [103]:

kmeans_prices = KMeans(n_clusters=4, random_state=0).fit(prices_std)
kmeans_prices.labels_[0:10]

array([1, 1, 0, 3, 0, 1, 0, 3, 0, 3])

Now let's add the label to our neighborhoods dataframe


In [105]:
madrid_prices['label'] = kmeans_prices.labels_

In [106]:
madrid_prices.head()


,Districto,neighborhood,Shape_Leng,Shape_Area,district_id,distNeigh_Id,code,neighborhood_id,Cod_distNeigh,latitude,longitude,Price 1,Price 2,Price 3,Price 4,label
0,Centro,Palacio,5754.822748,1469905.684,1,11,101,1,1_1,40.416841,-3.711605,13,32,25,0,1
1,Centro,Embajadores,4275.227681,1033724.698,1,12,102,2,1_2,40.409836,-3.702715,21,47,7,0,1
2,Centro,Cortes,3731.07903,591874.1219,1,13,103,3,1_3,40.414867,-3.697085,26,81,21,0,0
3,Centro,Justicia,3597.421427,739414.338,1,14,104,4,1_4,40.423301,-3.696126,22,74,9,1,3
4,Centro,Universidad,4060.075813,948027.0773,1,15,105,5,1_5,40.426313,-3.706528,37,70,16,0,0


<H2>4.Conclusion</H2>

<h3>4.1 Clustering by venue category</h3>
Using machine learning algorithms, we have grouped the neighborhoods of Madrid into 5 groups based on the category and number of restaurants.

<ul>
   <li>Cluster 0 comprises 30 neighborhoods, and are in areas surrounding the center of the city; they host the greatest ratio and total number of Spanish restaurants. It also has a higher proportion of breakfast restaurants and bakeries (these are generally working neighborhoods)</li>
<li>Cluster 1 comprises 30 neighborhoods, mostly in the south east of the city; they have the highest percentages of pizza places, fast food restaurants and Chinese restaurants.</li>
<li>Cluster 2 comprises 18 venues, mostly in the center of the city. They have a great proportion of tapas restaurants (very appreciated by tourists), and the greatest proportion of mexican restaurants.</li>
<li>Cluster 3 is the largest one, with 42 neighborhoods, mostly in the north and center of the city. They have a greater proportion, and a great total number, of Spanish restaurants, and a variety of other restaurants.</li>
<li>Cluster 4 comprises 11 neighborhoods, mostly in the south west of the city. They have the lowest number of restaurants, but they seem to have more variety of restaurants.</li>
</ul>

<h3>4.1 Clustering by venue price</h3>
Using the K-Means algorithm, we have clustered the neighborhoods in Madrid based on the number of restaurants of different prices in Madrid. 

<ul>
 <li>Cluster 0 comprises 30 neighborhoods. These neighborhoods have more expensive restaurants (prices 3 and 4)</li>
 <li>Cluster 1 comprises 48 neighborhoods. They have a similar number of restaurants of price 1 and 2, and a relatively low number of expensive restaurants.</li>
 <li>Cluster 2 comprises 23 neighborhoods, and have the lowest number of restaurants of all, very few of them in categories 3 and 4.</li>
 <li>Cluster 3 comprises 23 neighborhoods. They have the highest number of restaurants, with relatively a highest number of restaurants of category 2, and a total high number of restaurants of category 3.</li>
    </ul>